# <center> <img src="img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Carrera: <Coloca el nombre de tu carrera aqui>** </center>
---
### <center> **Primavera 2025** </center>
---

**Lab 07**

**Fecha**: 3 de abril
 2025

**Nombre del Equipo**: ArribaLinux

**Profesor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-ArribaLinux") \
    .master("spark://ac7f0d7e8e91:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/03 21:33:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Leer archivos como stream

In [3]:
from pyspark.sql.types import StructType, StringType
from pyspark.sql.functions import col


schema = StructType().add("value", StringType())
logs_df = spark.readStream \
    .schema(schema) \
    .text("/home/jovyan/notebooks/data/logs")


Parsear columnas del log 


In [4]:
from pyspark.sql.functions import split

parsed_logs = logs_df \
    .withColumn("timestamp", split(col("value"), " \\| ")[0]) \
    .withColumn("level", split(col("value"), " \\| ")[1]) \
    .withColumn("message", split(col("value"), " \\| ")[2]) \
    .withColumn("server", split(col("value"), " \\| ")[3])


Filtrar errores críticos (500)

In [5]:
critical_errors = parsed_logs \
    .filter(col("level") == "ERROR") \
    .filter(col("message").contains("500"))


Mostrar resultados

In [ ]:
query = critical_errors.writeStream \
    .outputMode("append") \
    .format("console") \
    .option("checkpointLocation", "/home/jovyan/notebooks/data/checkpoint_lab07") \
    .trigger(processingTime="10 seconds") \
    .start()

query.awaitTermination(15)


25/04/03 21:34:17 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/04/03 21:34:17 WARN StreamingQueryManager: Stopping existing streaming query [id=ce9d1f4b-ce85-433c-af5b-f91a6c1d7fa9, runId=3a89ee31-0de4-45fb-8569-59dc5dd68060], as a new run is being started.


-------------------------------------------
Batch: 2
-------------------------------------------
+--------------------+-------------------+-----+--------------------+-------------+
|               value|          timestamp|level|             message|       server|
+--------------------+-------------------+-----+--------------------+-------------+
|2025-04-03 $H:34:...|2025-04-03 $H:34:16|ERROR|500 Internal Serv...|server-node-2|
|2025-04-03 $H:34:...|2025-04-03 $H:34:16|ERROR|500 Internal Serv...|server-node-2|
|2025-04-03 $H:34:...|2025-04-03 $H:34:16|ERROR|500 Internal Serv...|server-node-1|
|2025-04-03 $H:34:...|2025-04-03 $H:34:16|ERROR|500 Internal Serv...|server-node-2|
|2025-04-03 $H:34:...|2025-04-03 $H:34:16|ERROR|500 Internal Serv...|server-node-3|
|2025-04-03 $H:34:...|2025-04-03 $H:34:16|ERROR|500 Internal Serv...|server-node-2|
|2025-04-03 $H:34:...|2025-04-03 $H:34:16|ERROR|500 Internal Serv...|server-node-3|
|2025-04-03 $H:34:...|2025-04-03 $H:34:16|ERROR|500 Internal Se

False

-------------------------------------------
Batch: 4
-------------------------------------------
+--------------------+-------------------+-----+--------------------+-------------+
|               value|          timestamp|level|             message|       server|
+--------------------+-------------------+-----+--------------------+-------------+
|2025-04-03 $H:34:...|2025-04-03 $H:34:36|ERROR|500 Internal Serv...|server-node-1|
|2025-04-03 $H:34:...|2025-04-03 $H:34:36|ERROR|500 Internal Serv...|server-node-3|
|2025-04-03 $H:34:...|2025-04-03 $H:34:36|ERROR|500 Internal Serv...|server-node-2|
|2025-04-03 $H:34:...|2025-04-03 $H:34:36|ERROR|500 Internal Serv...|server-node-2|
|2025-04-03 $H:34:...|2025-04-03 $H:34:36|ERROR|500 Internal Serv...|server-node-1|
|2025-04-03 $H:34:...|2025-04-03 $H:34:36|ERROR|500 Internal Serv...|server-node-1|
|2025-04-03 $H:34:...|2025-04-03 $H:34:36|ERROR|500 Internal Serv...|server-node-2|
|2025-04-03 $H:34:...|2025-04-03 $H:34:36|ERROR|500 Internal Se

In [17]:
query.stop()